In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt


In [2]:
# Download traning data from open datasets

training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor(),
)

## Download test data from open datasets.

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor(),
)

100%|██████████| 26421880/26421880 [00:01<00:00, 18728875.48it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 337038.41it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6073013.82it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 14214797.23it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
type(training_data)

torchvision.datasets.mnist.FashionMNIST

In [4]:
batch_size = 64

## Create data Loader

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X,y in test_dataloader:
  print("Shape of X [N,C, H, W]: ",X.shape)
  print("Shape of y: ",y.shape, y.dtype)

  #print(X)
  #print(y)

  break

Shape of X [N,C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [5]:
## Get cpu or gpu device for training.
device = 'cuda' if torch.cuda.is_available() else "cpu"
print(f"Using { device} device")

Using cpu device


In [6]:
## Define Model

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  def forward(self, X):
      X = self.flatten(X)
      logits = self.linear_relu_stack(X)
      return logits



model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
## Loss Function

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)



In [8]:
## Define Train Function

def train(Dataloader, model, loss_fn, optimizer):
  size = len(Dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(train_dataloader):
    X, y  = X.to(device), y.to(device)

    ## Cooperate prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    ## Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch* len(X)
      print(f"loss : {loss:>7f} [{current:>5d}/{size:>5d}]")


In [9]:
def test(dataloader, model, loss_fn):
  size = len(test_dataloader)
  num_batches = len(test_dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f" Test Error: \n Accuracy: {(correct):>0.1f} %, Avg loss: {test_loss:>8f} \n")

In [10]:
## compile the model

epochs = 5
for t in range(epochs):
  print(f"Epochs {t+1}\n----------------------------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done !")

Epochs 1
----------------------------------------------------
loss : 2.302153 [    0/60000]
loss : 2.285298 [ 6400/60000]
loss : 2.266679 [12800/60000]
loss : 2.271160 [19200/60000]
loss : 2.247707 [25600/60000]
loss : 2.219799 [32000/60000]
loss : 2.230861 [38400/60000]
loss : 2.194581 [44800/60000]
loss : 2.185634 [51200/60000]
loss : 2.158199 [57600/60000]
 Test Error: 
 Accuracy: 30.6 %, Avg loss: 2.153362 

Epochs 2
----------------------------------------------------
loss : 2.153881 [    0/60000]
loss : 2.146863 [ 6400/60000]
loss : 2.090520 [12800/60000]
loss : 2.113612 [19200/60000]
loss : 2.047276 [25600/60000]
loss : 1.990207 [32000/60000]
loss : 2.021420 [38400/60000]
loss : 1.939536 [44800/60000]
loss : 1.937651 [51200/60000]
loss : 1.862602 [57600/60000]
 Test Error: 
 Accuracy: 35.4 %, Avg loss: 1.869223 

Epochs 3
----------------------------------------------------
loss : 1.891631 [    0/60000]
loss : 1.868324 [ 6400/60000]
loss : 1.754620 [12800/60000]
loss : 1.797936 

In [11]:
## Save the model

torch.save(model.state_dict(), "model.pth")
print("Saved Pytorch Model State to model.pth")

Saved Pytorch Model State to model.pth


In [12]:
## Load The Model
## If we want to reuse our save model

model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [14]:
## Prediction

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
